# Animation of 3D models generated by spateo

In [1]:
import io, warnings, os
from pathlib import Path
import base64
from IPython.display import HTML

import spateo as st
warnings.filterwarnings('ignore')

## Load the data

In [2]:
os.chdir(f"/media/pc001/Yao/Projects/Project_spateo/mouse_heart")
out_image_path = f"animations/drosophila_midgut_models"
Path(out_image_path).mkdir(parents=True, exist_ok=True)

In [3]:
cpo = [(41, 1209, 57), (13, 8, -3), (0, 0, 1)]
 
sample_id = "E7_8h"
germ_adata = st.read_h5ad(f"/media/pc001/Yao/Projects/Project_drosophila/Data_v4/migration-hotspot/drosophila_E7_8h_germ_layer_new/h5ad/{sample_id}_germband_v3.h5ad")
tissue_adata = germ_adata[germ_adata.obs["anno_tissue_new"] == "midgut", :]
tissue_adata.X = tissue_adata.layers["X_counts"].copy()
tissue_adata = tissue_adata[tissue_adata.X.sum(axis=1)!=0, tissue_adata.X.sum(axis=0)!=0]
tissue_pc, _ = st.tdr.construct_pc(
    adata=tissue_adata.copy(),
    spatial_key="3d_align_spatial",
    groupby="anno_tissue_new",
    key_added="tissue",
    colormap={"amnioserosa": "#bdcf32", "CNS": "#ea5545", "hindgut": "#27aeef", "midgut": "#87bc45", "muscle": "#f46a9b", "salivary gland": "#ef9b20",},
)
tissue_adata

## Animation of mapping model

In [4]:
import numpy as np
stage2_germ_adata = st.read_h5ad(f"/media/pc001/Yao/Projects/Project_drosophila/Data_v4/migration-hotspot/drosophila_E8_9h_germ_layer/h5ad/E8_9h_germ_layer_v3.h5ad")
stage2_tissue_adata = stage2_germ_adata[stage2_germ_adata.obs["anno_tissue_new"] == "midgut", :]
stage2_tissue_adata.X = stage2_tissue_adata.layers["X_counts"].copy()
stage2_tissue_pc, _ = st.tdr.construct_pc(
    adata=stage2_tissue_adata.copy(),
    spatial_key="3d_align_spatial",
    groupby="anno_tissue_new",
    key_added="tissue",
    colormap={"amnioserosa": "#bdcf32", "CNS": "#ea5545", "hindgut": "#27aeef", "midgut": "#87bc45", "muscle": "#f46a9b", "salivary gland": "#ef9b20",},
)
models_distance = np.asarray([0, 0, -260])
stage2_tissue_pc.points = stage2_tissue_pc.points + models_distance
stage2_tissue_adata

In [5]:
model_lines, _ = st.tdr.construct_align_lines(
            model1_points=tissue_adata.obsm["3d_align_spatial"].copy(),
            model2_points=tissue_adata.obsm["X_cells_mapping"] + models_distance,
            key_added="mapping",
            label="mapping lines",
            color="gainsboro",
            alpha=1.0,
        )

In [7]:
mapping_cpo = [(18, 1370, -141),(-1.8, 5.8, -156.8), (0, 0, 1)]
st.pl.three_d_plot(
    model=st.tdr.collect_models([tissue_pc, stage2_tissue_pc, model_lines]),
    key=["tissue", "tissue", "mapping"],
    filename=os.path.join(out_image_path, f"midgut_mapping_model_animation.mp4"),
    model_size=[14, 14, 2],
    model_style=["points", "points", "wireframe"],
    opacity=[1, 1, 0.1],
    show_legend=True,
    off_screen=False,
    jupyter="static",
    background="white",
    window_size=(2560, 2048),
    cpo=mapping_cpo,
    text=f"\n3D alignment of midgut (Drosophila S11&S13)",
    text_kwargs={"font_size":20},
    view_up=(0, 0, 1),
    framerate=6
)

## Animation of trajectory model

In [8]:
st.tdr.morphofield_sparsevfc(
    adata=tissue_adata,
    spatial_key="3d_align_spatial",
    V_key="V_cells_mapping",
    key_added="VecFld_morpho",
    NX=np.asarray(tissue_adata.obsm['3d_align_spatial']),
    inplace=True,
)

In [10]:
st.tdr.morphopath(
    adata=tissue_adata,
    # layer="log1p_X",
    vf_key="VecFld_morpho",
    key_added="fate_morpho",
    t_end=10000,
    interpolation_num=50,
    cores=20
)

In [11]:
trajectory_model, _ = st.tdr.construct_trajectory(
    adata=tissue_adata,
    fate_key="fate_morpho",
    n_sampling=800,
    sampling_method="trn",
    key_added="obs_index",
    label=np.asarray(tissue_adata.obs.index), # stage1_tissue_adata.uns["VecFld_morpho"]["V"][:, 2].flatten(),
)

tissue_adata.obs["V_x"] = tissue_adata.uns["VecFld_morpho"]["V"][:, 0].flatten()
st.tdr.add_model_labels(
        model=trajectory_model,
        key_added="V_x",
        labels=np.asarray(tissue_adata[np.asarray(trajectory_model.point_data["obs_index"])].obs["V_x"]),
        colormap="Spectral",
        where="point_data",
        inplace=True,
    )

In [12]:
cells_models, _ = st.tdr.construct_genesis(
    adata=tissue_adata,
    fate_key="fate_morpho",
    n_steps = 100,
    logspace=True,
    t_end=10000,
    label=[tissue_adata.uns["VecFld_morpho"]["V"][:, 0]] * 100,
    color="Spectral"
)

In [13]:
st.pl.three_d_animate(
    models=cells_models,
    stable_model=trajectory_model,
    key="genesis",
    stable_kwargs=dict(
        key="V_x",
        model_style="wireframe",
        model_size=5,
        opacity=0.5,
        colormap="Spectral",
        show_legend=False,
    ),
    filename=os.path.join(out_image_path, f"midgut_morphofield_model_animation.mp4"),
    colormap="Spectral",
    model_style="points",
    model_size=14,
    jupyter="static",
    background="white",
    text=f"\nPredicted migration trajectories of midgut (Drosophila S11)",
    text_kwargs={"font_size":20},
    window_size=(2560, 2048),
    cpo=cpo,
    framerate=6)

## Merge all animations

In [14]:
st.pl.merge_animations(
    mp4_files=[
        os.path.join(out_image_path, f"midgut_mapping_model_animation.mp4"),
        os.path.join(out_image_path, f"midgut_morphofield_model_animation.mp4"),
    ],
    filename=os.path.join(out_image_path, f"Drosophila_midgut_animation.mp4"),
)